In [5]:
from strands import Agent
from strands.models.anthropic import AnthropicModel
from strands.models.ollama import OllamaModel
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import os

load_dotenv()

anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")


In [3]:
ollama_model = OllamaModel(model_id="qwen3:8b", host="http://localhost:11434")
anthropic_model = AnthropicModel(
    client_args={
        "api_key": anthropic_api_key,
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.1,
    }
)

In [6]:
sample_agent = Agent(model=ollama_model, system_prompt="Extract the name of the person and the address")

class Info(BaseModel):
    name: str = Field(description="The name of the person")
    door_no: str = Field(description="The door number of the person")
    street: str = Field(description="The street name of the person")
    city: str = Field(description="The city of the person")
    province: str = Field(description="The province of the person")
    country: str = Field(description="The country of the person")
    

In [7]:
result = sample_agent.structured_output(Info,"srivatsa lives in 50 graydon hall drive, north york, ontario, canada")

In [8]:
type(result)

__main__.Info

In [10]:
result.model_dump()

{'name': 'Srivatsa',
 'door_no': '50',
 'street': 'graydon hall drive',
 'city': 'north york',
 'province': 'ontario',
 'country': 'canada'}

In [11]:
result = sample_agent("srivatsa lives in 50 graydon hall drive, north york, ontario, canada")
type(result)


<think>
Okay, let's see. The user wants me to extract the name of the person and their address from the given text. The input is "srivatsa lives in 50 graydon hall drive, north york, ontario, canada" repeated twice. 

First, I need to identify the person's name. The text starts with "srivatsa," which looks like a first name. Since there's no other name mentioned, I can assume that's the person's name.

Next, the address part. The address is "50 graydon hall drive, north york, ontario, canada." I need to parse this into standard address components. Typically, an address includes street number, street name, city, state, and country. Here, "50 graydon hall drive" is the street address. Then "north york" is the city. "Ontario" is the province, and "canada" is the country. 

Wait, but sometimes "North York" is a city in Ontario, Canada. So the address components should be structured as: Street: 50 Graydon Hall Drive, City: North York, Province: Ontario, Country: Canada. 

I should make sure

strands.agent.agent_result.AgentResult

In [14]:
result

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '<think>\nOkay, let\'s see. The user wants me to extract the name of the person and their address from the given text. The input is "srivatsa lives in 50 graydon hall drive, north york, ontario, canada" repeated twice. \n\nFirst, I need to identify the person\'s name. The text starts with "srivatsa," which looks like a first name. Since there\'s no other name mentioned, I can assume that\'s the person\'s name.\n\nNext, the address part. The address is "50 graydon hall drive, north york, ontario, canada." I need to parse this into standard address components. Typically, an address includes street number, street name, city, state, and country. Here, "50 graydon hall drive" is the street address. Then "north york" is the city. "Ontario" is the province, and "canada" is the country. \n\nWait, but sometimes "North York" is a city in Ontario, Canada. So the address components should be structured as: Stree